In [1]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [1]:
# Importing Datasets
path=r'C:\Users\manis\Documents\Instacart Basket Analysis'

In [3]:
# Importing Customer dataset
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [4]:
df.shape

(206209, 10)

In [5]:
df.head(10)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


In [6]:
df.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


##### Data Wrangling

In [7]:
# Renaming the columns

df = df.rename(columns = {'First Name':'first_name', 'Surnam':'last_name', 'Gender':'gender', 'STATE':'state', 'Age':'age','n_dependants':'no_of_dependants','fam_status':'family_status'})


In [8]:
df.dtypes

user_id              int64
first_name          object
last_name           object
gender              object
state               object
age                  int64
date_joined         object
no_of_dependants     int64
family_status       object
income               int64
dtype: object

##### Data Quality and Consistency Checks

In [9]:
#Checking for and addressing Values
df.isnull().sum()

user_id                 0
first_name          11259
last_name               0
gender                  0
state                   0
age                     0
date_joined             0
no_of_dependants        0
family_status           0
income                  0
dtype: int64

##### first_name is the only column with missing values for 11259 rows. To address this issue , I would not drop this column because these missing values are just the 5% of the total dataset. We can't impute the values for this column because its datatype is Object. I can create a subset here which can have only the clean data 

In [10]:
# Creating subset dataframe with clean data, no missing values
df_clean = df[df['first_name'].isnull() == False]

In [11]:
df.shape

(206209, 10)

In [12]:
df_clean.shape


(194950, 10)

##### My new dataframe do not have 11259 rows with the missing first_name column data and going forward I will use the clean dataframe

##### Finding duplicates

In [13]:
df_dups = df_clean[df_clean.duplicated()]

In [14]:
df_dups

,user_id,first_name,last_name,gender,state,age,date_joined,no_of_dependants,family_status,income


##### Above executed code shows that there are no duplicates in the df_clean dataframe

##### Checking for mixed datatypes


In [15]:
for col in df_clean.columns.tolist():
  weird = (df_clean[[col]].applymap(type) != df_clean[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_clean[weird]) > 0:
    print (col)
    
    

##### No mixed datatypes

In [16]:
# Combining Customer data with prepared Instacart data
# Importing Prepared Instacart Data
df_ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', '4.8_ords_prods_merge.pkl'))

In [17]:
df_ords_prods.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,ordered_today,product_id,add_to_cart_order,...,aisle_id,department_id,prices,_merge,max_order,loyalty_flag,spending,spending_flag,Order_frequency_median,order_frequency_flag
0,2539329,1,prior,1,2,8,NaN,True,196,1,...,77.0,7.0,9.0,both,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2539329,1,prior,1,2,8,NaN,True,14084,2,...,91.0,16.0,12.5,both,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,2539329,1,prior,1,2,8,NaN,True,12427,3,...,23.0,19.0,4.4,both,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2539329,1,prior,1,2,8,NaN,True,26088,4,...,23.0,19.0,4.7,both,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,2539329,1,prior,1,2,8,NaN,True,26405,5,...,54.0,17.0,1.0,both,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [18]:
df_clean.head()

,user_id,first_name,last_name,gender,state,age,date_joined,no_of_dependants,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


##### I will merge Customer data with prepared Instacart data using user_id as the common key for both the dataframes

##### I have to drop the column _merge from df_ords_prods dataframe before doing the merger with customer data

##### I got a memory error while merging the customer dataset with Instacart dataset. To fix this , I will try to change datatypes for few of the columns from both the datasets.Before doing this, I will check the max value for the columns and then I will write a code to change their datatype with low memory usage.

In [19]:
df_ords_prods.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32435059 entries, 0 to 32435058
Data columns (total 22 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   eval_set                object  
 3   order_number            int64   
 4   order_dow               int64   
 5   order_hour_of_day       int64   
 6   days_since_prior_order  float64 
 7   ordered_today           bool    
 8   product_id              int64   
 9   add_to_cart_order       int64   
 10  reordered               int64   
 11  product_name            object  
 12  aisle_id                float64 
 13  department_id           float64 
 14  prices                  float64 
 15  _merge                  category
 16  max_order               int64   
 17  loyalty_flag            object  
 18  spending                float64 
 19  spending_flag           object  
 20  Order_frequency_median  float64 
 21  order_

In [20]:
df_ords_prods['Order_frequency_median'].max()

30.0

In [21]:
df_ords_prods['Order_frequency_median'] = df_ords_prods['Order_frequency_median'].astype('float16')

In [22]:
df_ords_prods['spending'].max()

25005.425

In [23]:
df_ords_prods['spending'] = df_ords_prods['spending'].astype('float16')

In [24]:
df_ords_prods['max_order'].max()

99

In [25]:
df_ords_prods['max_order'] = df_ords_prods['max_order'].astype('int8')

In [26]:
df_ords_prods['aisle_id'].max()

134.0

In [27]:
df_ords_prods['aisle_id'] = df_ords_prods['aisle_id'].astype('float16')

In [28]:
df_ords_prods['add_to_cart_order'].max()

145

In [29]:
df_ords_prods['add_to_cart_order'] = df_ords_prods['add_to_cart_order'].astype('int8')

In [30]:
df_ords_prods.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,ordered_today,product_id,add_to_cart_order,...,aisle_id,department_id,prices,_merge,max_order,loyalty_flag,spending,spending_flag,Order_frequency_median,order_frequency_flag
0,2539329,1,prior,1,2,8,NaN,True,196,1,...,77.0,7.0,9.0,both,10,New customer,6.367188,Low spender,20.5,Non-frequent customer
1,2539329,1,prior,1,2,8,NaN,True,14084,2,...,91.0,16.0,12.5,both,10,New customer,6.367188,Low spender,20.5,Non-frequent customer
2,2539329,1,prior,1,2,8,NaN,True,12427,3,...,23.0,19.0,4.4,both,10,New customer,6.367188,Low spender,20.5,Non-frequent customer
3,2539329,1,prior,1,2,8,NaN,True,26088,4,...,23.0,19.0,4.7,both,10,New customer,6.367188,Low spender,20.5,Non-frequent customer
4,2539329,1,prior,1,2,8,NaN,True,26405,5,...,54.0,17.0,1.0,both,10,New customer,6.367188,Low spender,20.5,Non-frequent customer


In [31]:
df_ords_prods['spending_flag'] = df_ords_prods['spending_flag'].astype('category')

In [32]:
df_ords_prods['order_frequency_flag'] = df_ords_prods['order_frequency_flag'].astype('category')

In [33]:
df_ords_prods['loyalty_flag'] = df_ords_prods['loyalty_flag'].astype('category')

In [34]:
df_ords_prods = df_ords_prods.drop('eval_set', 1)

In [35]:
df_ords_prods['days_since_prior_order'].max()

30.0

In [36]:
df_ords_prods['days_since_prior_order'] = df_ords_prods['days_since_prior_order'].astype('float16')

In [37]:
df_ords_prods['order_hour_of_day'].max()

23

In [38]:
df_ords_prods['order_hour_of_day'] = df_ords_prods['order_hour_of_day'].astype('int8')

In [39]:
df_ords_prods.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32435059 entries, 0 to 32435058
Data columns (total 21 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   order_number            int64   
 3   order_dow               int64   
 4   order_hour_of_day       int8    
 5   days_since_prior_order  float16 
 6   ordered_today           bool    
 7   product_id              int64   
 8   add_to_cart_order       int8    
 9   reordered               int64   
 10  product_name            object  
 11  aisle_id                float16 
 12  department_id           float64 
 13  prices                  float64 
 14  _merge                  category
 15  max_order               int8    
 16  loyalty_flag            category
 17  spending                float16 
 18  spending_flag           category
 19  Order_frequency_median  float16 
 20  order_frequency_flag    category
dtypes: boo

In [40]:
df_ords_prods['order_dow'].max()

6

In [41]:
df_ords_prods['order_dow'] = df_ords_prods['order_dow'].astype('int8')

In [42]:
df_ords_prods.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32435059 entries, 0 to 32435058
Data columns (total 21 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   order_number            int64   
 3   order_dow               int8    
 4   order_hour_of_day       int8    
 5   days_since_prior_order  float16 
 6   ordered_today           bool    
 7   product_id              int64   
 8   add_to_cart_order       int8    
 9   reordered               int64   
 10  product_name            object  
 11  aisle_id                float16 
 12  department_id           float64 
 13  prices                  float64 
 14  _merge                  category
 15  max_order               int8    
 16  loyalty_flag            category
 17  spending                float16 
 18  spending_flag           category
 19  Order_frequency_median  float16 
 20  order_frequency_flag    category
dtypes: boo

In [43]:
df_ords_prods['reordered'].max()

1

In [44]:
df_ords_prods['reordered'] = df_ords_prods['reordered'].astype('int8')

In [45]:
df_ords_prods.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32435059 entries, 0 to 32435058
Data columns (total 21 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   order_number            int64   
 3   order_dow               int8    
 4   order_hour_of_day       int8    
 5   days_since_prior_order  float16 
 6   ordered_today           bool    
 7   product_id              int64   
 8   add_to_cart_order       int8    
 9   reordered               int8    
 10  product_name            object  
 11  aisle_id                float16 
 12  department_id           float64 
 13  prices                  float64 
 14  _merge                  category
 15  max_order               int8    
 16  loyalty_flag            category
 17  spending                float16 
 18  spending_flag           category
 19  Order_frequency_median  float16 
 20  order_frequency_flag    category
dtypes: boo

##### After doing all the changes in datatypes for different columns , memory usage for the dataframe df_ords_prods reduced to 2.5 GB from 5.1 GB, I will try to merge again this dataframe with customer dataframe df_clean

In [47]:
df_ords_prods.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,ordered_today,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,_merge,max_order,loyalty_flag,spending,spending_flag,Order_frequency_median,order_frequency_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,...,77.0,7.0,9.0,both,10,New customer,6.367188,Low spender,20.5,Non-frequent customer
1,2539329,1,1,2,8,NaN,True,14084,2,0,...,91.0,16.0,12.5,both,10,New customer,6.367188,Low spender,20.5,Non-frequent customer
2,2539329,1,1,2,8,NaN,True,12427,3,0,...,23.0,19.0,4.4,both,10,New customer,6.367188,Low spender,20.5,Non-frequent customer
3,2539329,1,1,2,8,NaN,True,26088,4,0,...,23.0,19.0,4.7,both,10,New customer,6.367188,Low spender,20.5,Non-frequent customer
4,2539329,1,1,2,8,NaN,True,26405,5,0,...,54.0,17.0,1.0,both,10,New customer,6.367188,Low spender,20.5,Non-frequent customer


In [48]:
# Dropping the _merge column
df_ords_prods = df_ords_prods.drop('_merge', 1)

In [49]:
# Merging Instacart data with Customer Data
df_merge = df_ords_prods.merge(df_clean, on = 'user_id', indicator = True)


In [50]:
df_merge.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,ordered_today,product_id,add_to_cart_order,reordered,...,first_name,last_name,gender,state,age,date_joined,no_of_dependants,family_status,income,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2539329,1,1,2,8,NaN,True,14084,2,0,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,2539329,1,1,2,8,NaN,True,12427,3,0,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2539329,1,1,2,8,NaN,True,26088,4,0,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,2539329,1,1,2,8,NaN,True,26405,5,0,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [51]:
df_merge['_merge'].value_counts(dropna = False)

both          30658154
left_only            0
right_only           0
Name: _merge, dtype: int64

##### Merge is successful

In [53]:
df_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', '4.9_Part1_ords_prods_cust.pkl'))